We will clean the csv file which has historical data.

KD line normalization
Z(1.5): 𝐾𝑡−1 < 𝐷𝑡−1,𝐷𝑡 < 𝐾𝑡, and (20 >= 𝐾𝑡−2 and 20 >=𝐾𝑡−1 and 25 <= 𝐾𝑡)
𝐴(1): 𝐾𝑡−1 < 𝐷𝑡−1,𝐷𝑡 < 𝐾𝑡, and (20 < 𝐾𝑡−2 < 80 or 20 < 𝐾𝑡−1 < 80 or 20 < 𝐾𝑡 < 80)
𝐵(-1.5): 𝐾𝑡−1 > 𝐷𝑡−1,𝐷𝑡 > 𝐾𝑡 and (𝐾𝑡−2 >= 80 and 𝐾𝑡−1 >= 80 and 𝐾𝑡 <= 75)
𝐵(-1): 𝐾𝑡−1 > 𝐷𝑡−1,𝐷𝑡 > 𝐾𝑡 and (20 < 𝐾𝑡−2 < 80 or 20 < 𝐾𝑡−1 < 80 or 20 < 𝐾𝑡 < 80)
𝐶(0): Otherwise

MACD normalization:
𝐴(1):𝐷𝐼𝐹𝑡−1 < 9𝑀𝐴𝐶𝐷𝑡−1 𝑎𝑛𝑑 9𝑀𝐴𝐶𝐷𝑡 < 𝐷𝐼𝐹𝑡
𝐵(-1): 𝐷𝐼𝐹𝑡−1 > 9𝑀𝐴𝐶𝐷𝑡−1 𝑎𝑛𝑑 9𝑀𝐴𝐶𝐷𝑡 > 𝐷𝐼𝐹𝑡
𝐶(0): Otherwise

Label Design 
a) Daily
1. Trend up: 𝐶𝑙𝑜𝑠𝑒𝑡+1 -𝐶𝑙𝑜𝑠𝑒𝑡 > 𝐶𝑙𝑜𝑠𝑒𝑡 × α
2. Flat trend: 𝐶𝑙𝑜𝑠𝑒𝑡 × α > 𝐶𝑙𝑜𝑠𝑒𝑡+1 -𝐶𝑙𝑜𝑠𝑒𝑡 >−𝐶𝑙𝑜𝑠𝑒𝑡 × α
3. Trend down: 𝐶𝑙𝑜𝑠𝑒𝑡+1 -𝐶𝑙𝑜𝑠𝑒𝑡 < −𝐶𝑙𝑜𝑠𝑒𝑡 × α
b) Short Term (use moving average)

NOTE:
1) for all 3 column of macd line ta have values, 33 period needed
2) for all 2 column of stochastic kd line ta have values, 16 period needed

In [94]:
import pandas as pd
import numpy as np
import pandas_ta as ta
from sklearn.preprocessing import MinMaxScaler
import logging


# NOTES: We are currenlty assuming that all values are present in the excel file:
# the data is continous. TO ADD: check for discontnous data and find a replacement strategy.


class ETL:
    def __init__(self):
        print('created')

    def Load_Clean_Data(self, input_file_path, interval='1d'):
        try:
            # second row is column names
            df = pd.read_csv(input_file_path, header=1)
            col_list = ['unix_start_datetime', 'start_date', 'symbol', 'open',
                        'high', 'low', 'close', 'base_volume', 'quote_volume', 'num_trades']
            df.columns = col_list
            df["interval"] = interval
            df.dropna(subset=['symbol', 'unix_start_datetime', 'open','close', 'high', 'low', 'base_volume'], inplace=True)
            return df.copy()
        except Exception as e:
            logging.error('Failed to load and clean data: \n' + str(e))

    def Add_TI_Data(self, df, k=14, d=3, cols= ['ti_stoch_kd','ti_MACD']):
        try:
            # add technical indicators
            # adds 2 columns "STOCHk_14_3_3","STOCHd_14_3_3"
            df.ta.stoch(high='high', low='low', close='close',k=k, d=d, fillna=True, append=True)
            MACD = ta.macd(df['close'])
            df['MACD'] = MACD['MACD_12_26_9']
            df['MACDs'] = MACD['MACDs_12_26_9']
            df = df.rename(columns={"STOCHk_14_3_3": "STOCHk", "STOCHd_14_3_3": "STOCHd"})
            #Normalize data
            norm_kd=[0,0]
            norm_macd=[0,0]
            for index, row in df.iloc[2:].iterrows():
                # KD line normalization
                if (df.at[index - 1,'STOCHk'] < df.at[index - 1,'STOCHd'] and row['STOCHk'] < row['STOCHd']):
                    if (20 >= df.at[index - 2,'STOCHk'] and 20 >= df.at[index - 1,'STOCHk'] and 25 <= row['STOCHk']):
                        norm_kd.append(1.5)
                    elif (20 < df.at[index - 2,'STOCHk'] < 80 or 20 < df.at[index - 1,'STOCHk'] < 80 or 20 < row['STOCHk'] < 80):
                        norm_kd.append(1)
                    else:
                        norm_kd.append(0)
                elif (df.at[index - 1,'STOCHk'] > df.at[index - 1,'STOCHd'] and row['STOCHk'] > row['STOCHd']):
                    if (80 <= df.at[index - 2,'STOCHk'] and 80 <= df.at[index - 1,'STOCHk'] and 75 >= row['STOCHk']):
                        norm_kd.append(-1.5)
                    elif (20 < df.at[index - 1,'STOCHk'] < 80 or 20 < df.at[index - 1,'STOCHk'] < 80 or 20 < row['STOCHk'] < 80):
                        norm_kd.append(-1)
                    else:
                        norm_kd.append(0)
                else:
                    norm_kd.append(0)

                # MACD Norlmalization
                if (df.at[index - 1,'MACD'] < df.at[index - 1,'MACDs'] and row['MACDs'] < row['MACD']):
                    norm_macd.append(1)
                elif (df.at[index - 1,'MACD'] > df.at[index - 1,'MACDs'] and row['MACDs'] > row['MACD']):
                    norm_macd.append(-1)
                else:
                    norm_macd.append(0)
            # print(df.shape[0])
            # print(len(norm_kd))
            # print(len(norm_macd))
            df[cols[0]]= norm_kd
            df[cols[1]]= norm_macd
            return df
        except Exception as e:
            logging.ERROR('Failed to add TI data: \n' + str(e))

    def Add_Label(self, df, window_size=5, alpha=0.005,cols_to_normalize= ['open','high', 'low', 'close', 'base_volume', 'quote_volume', 'num_trades']):
        try:
            #first normalize the ohlcv data
            # cols_to_normalize = ['open','high', 'low', 'close', 'base_volume', 'quote_volume', 'num_trades']
            df_norm = self.normalize_ohlcv_data(df,cols_to_normalize)
            for col in cols_to_normalize:
                df[col]= df_norm[col]
            #Create Labels: 2 types of label, a) short term b) daily
            ma_minus = df['close'].rolling(window=window_size).mean()
            ma_plus = ma_minus.shift(-(window_size-1))
            n=5
            resultA = []
            resultB = []
            for i in range(ma_plus.shape[0]):
                #NOTE: bull, bear flat or buy sell hold
                #TODO: what
                tempA = "up" if (ma_plus[i]-ma_minus[i]) > (ma_minus[i]*alpha) else "down" if (ma_plus[i]-ma_minus[i]) < (-ma_minus[i]*alpha) else "flat"
                resultA.append(tempA)
                if(i==(ma_plus.shape[0] -1)):
                    resultB.append('flat')
                else:
                    tempB= "up" if (df.loc[i+1,"close"]- df.loc[i,"close"]) > (df.loc[i,"close"]*alpha) else "down" if (df.loc[i+1,"close"]-  df.loc[i,"close"]) < (-df.loc[i,"close"]*alpha) else "flat"
                    resultB.append(tempB)
            df['result_A'] = resultA
            df['result_B'] = resultB
            #Drop all NA value
            df = df.drop(range(33)) #TODO: make dynamic
            return df
        except Exception as e:
            logging.error('Failed to add labels to data: \n' + str(e))
            return
    #Note we have created TI columns
    def normalize_ohlcv_data(self, df,cols_to_normalize):
        scaler = MinMaxScaler()
        scaler.fit(df[cols_to_normalize])
        df_norm = scaler.transform(df[cols_to_normalize])
        df_norm = pd.DataFrame(df_norm, columns=cols_to_normalize)
        return df_norm







In [95]:
filePth= './historical_data/Binance_ETHUSDT_d.csv'
etl =  ETL()
df_temp = etl.Load_Clean_Data(filePth)
cols= ['ti_stoch_kd','ti_MACD']
cols_to_normalize = ['open','high', 'low', 'close', 'base_volume', 'quote_volume', 'num_trades']
df=etl.Add_TI_Data(df_temp,cols=cols)
df=etl.Add_Label(df,cols_to_normalize=cols_to_normalize)
req_cols = cols_to_normalize+cols+['unix_start_datetime','result_A','result_B']
# df_temp_2= etl.Add_Label_TI_Data(df_temp.copy())
final_df=df[req_cols]
print(final_df)

created
      unix_start_datetime  start_date   symbol      open      high       low  \
33          1677196800000  2023-02-24  ETHUSDT  0.331646  0.330266  0.322500   
34          1677110400000  2023-02-23  ETHUSDT  0.330086  0.333264  0.333965   
35          1677024000000  2023-02-22  ETHUSDT  0.333604  0.330744  0.326694   
36          1676937600000  2023-02-21  ETHUSDT  0.342811  0.341137  0.335595   
37          1676851200000  2023-02-20  ETHUSDT  0.337831  0.341813  0.338674   
...                   ...         ...      ...       ...       ...       ...   
2046        1503273600000  2017-08-21  ETHUSDT  0.045584  0.054677  0.045942   
2047        1503187200000  2017-08-20  ETHUSDT  0.043533  0.045062  0.043406   
2048        1503100800000  2017-08-19  ETHUSDT  0.044359  0.044930  0.042359   
2049        1503014400000  2017-08-18  ETHUSDT  0.046198  0.047416  0.043641   
2050        1502928000000  2017-08-17  ETHUSDT  0.046014  0.047497  0.046676   

         close  base_volume  qu

/var/folders/36/f2w97nw92131s9_c7z1n0p900000gn/T/ipykernel_68851/224174429.py:34: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  MACD = ta.macd(df['close'])


In [87]:
print(df_temp_2['ti_kd'])
print(df_temp_2['ti_MACD'])

33      0
34      1
35      1
36      1
37      0
       ..
2046   -1
2047   -1
2048    0
2049    1
2050    1
Name: ti_kd, Length: 2018, dtype: int64
33      0
34      0
35      0
36      0
37      0
       ..
2046    0
2047    0
2048    0
2049    0
2050    0
Name: ti_MACD, Length: 2018, dtype: int64


In [35]:
import pandas as pd
import pandas_ta as ta
import numpy as np
df= pd.read_csv('./historical_data/Binance_ETHUSDT_d.csv',header=1) # second row is column names
col_list = ['unix_start_datetime','start_date','symbol','open','high','low','close','base_volume','quote_volume','num_trades']
df.columns = col_list
df.ta.stoch(high='high', low='low', close='close', k=14, d=3, fillna=True, append=True)
# df = df[["STOCHk_14_3_3","STOCHd_14_3_3"]].replace(True, np.nan)

# as we get Nan values in the first 16 rows, we drop these values as we have ample data
# display(df[["STOCHk_14_3_3","STOCHd_14_3_3"]].head(20))

#adds technical indicators
#adds 2 columns "STOCHk_14_3_3","STOCHd_14_3_3"
df.ta.stoch(high='high', low='low', close='close', k=14, d=3, fillna=True, append=True)
MACD = ta.macd(df['close'])
df['MACD']=MACD['MACD_12_26_9']
df['MACDs']=MACD['MACDs_12_26_9']
#rename the columns
df= df.rename(columns={"STOCHk_14_3_3":"STOCHk","STOCHd_14_3_3":"STOCHd"})

#Create Labels: 2 types of label, a) short term b) daily
window_size=5
ma_minus = df['Price'].rolling(window=window_size).mean()
ma_plus = ma_minus.shift(-(window_size-1))
alpha = 0.005
n=5
resultA = []
resultB = []

for i in range(ma_plus.shape[0]):
    #NOTE: bull, bear flat or buy sell hold
    #TODO: what
    resultA[i]= "up" if (ma_plus[i]-ma_minus[i]) > (ma_minus[i]*alpha) else "down" if (ma_plus[i]-ma_minus[i]) < (-ma_minus[i]*alpha) else "flat"
    if(i==(ma_plus.shape[0] -1)):
        resultB[i]='flat'# Maybe a better way
    else:
        resultB[i]= "up" if (df.loc[i+1,"close"]- df.loc[i,"close"]) > (df.loc[i,"close"]*alpha) else "down" if (df.loc[i+1,"close"]-  df.loc[i,"close"]) < (-df.loc[i,"close"]*alpha) else "flat"

#Note: Dropping N/A values
df.drop([range(33)])

#Normalize data
for index, row in df.iloc[2:].iterrows():
    # KD line normalization
    if(df['STOCHk',index - 1] < df['STOCHd',index - 1] and row['STOCHk'] < row['STOCHd']):
        if( 20>= df['STOCHk',index - 2] and 20>= df['STOCHk',index - 1] and 25 <= row['STOCHk']):
            df['ti_STOCHkd',index]= 1.5
        elif( 20< df['STOCHk',index - 2]<80 or 20< df['STOCHk',index - 1]<80 or 20<row['STOCHk']<80):
            df['ti_STOCHkd',index]= 1
    elif(df['STOCHk',index - 1] > df['STOCHd',index - 1] and row['STOCHk'] > row['STOCHd']):
        if( 80 <= df['STOCHk',index - 2] and 80 <= df['STOCHk',index - 1] and 75 >= row['STOCHk']):
            df['ti_STOCHkd',index]= -1.5
        elif( 20 < df['STOCHk',index - 2]< 80 or 20 < df['STOCHk',index - 1]< 80 or 20 < row['STOCHk']< 80):
            df['ti_STOCHkd',index]= -1
    else:
        df['ti_STOCHkd',index]= 0

    #MACD Norlmalization
    if(df['MACD',index - 1] < df['MACDs',index - 1] and row['MACDs'] < row['MACD']):
        df['ti_MACD',index] = 1
    elif(df['MACD',index - 1] > df['MACDs',index - 1] and row['MACDs'] > row['MACD']):
        df['ti_MACD',index] = -1
    else:
        df['ti_MACD',index] = 0

# KD line normalization
# Z(1.5): 𝐾𝑡−1 < 𝐷𝑡−1,𝐷𝑡 < 𝐾𝑡, and (20 >= 𝐾𝑡−2 and 20 >= 𝐾𝑡−1 and 25 <= 𝐾𝑡)
# 𝐴(1): 𝐾𝑡−1 < 𝐷𝑡−1,𝐷𝑡 < 𝐾𝑡, and (20 < 𝐾𝑡−2 < 80 or 20 < 𝐾𝑡−1 < 80 or 20 < 𝐾𝑡 < 80)
# 𝐵(-1.5): 𝐾𝑡−1 > 𝐷𝑡−1,𝐷𝑡 > 𝐾𝑡 and (𝐾𝑡−2 >= 80 and 𝐾𝑡−1 >= 80 and 𝐾𝑡 <= 75)
# C(-1): 𝐾𝑡−1 > 𝐷𝑡−1,𝐷𝑡 > 𝐾𝑡 and (20 < 𝐾𝑡−2 < 80 or 20 < 𝐾𝑡−1 < 80 or 20 < 𝐾𝑡 < 80)
# D(0): Otherwise

# MACD normalization:
# 𝐴(1):𝐷𝐼𝐹𝑡−1 < 9𝑀𝐴𝐶𝐷𝑡−1 𝑎𝑛𝑑 9𝑀𝐴𝐶𝐷𝑡 < 𝐷𝐼𝐹𝑡
# 𝐵(-1): 𝐷𝐼𝐹𝑡−1 > 9𝑀𝐴𝐶𝐷𝑡−1 𝑎𝑛𝑑 9𝑀𝐴𝐶𝐷𝑡 > 𝐷𝐼𝐹𝑡
# 𝐶(0): Otherwise


SyntaxError: invalid syntax (3493086906.py, line 33)

In [14]:
df = pd.DataFrame({'Price': [10, 12, 8, 14, 16, 11, 9, 13, 15, 17]})
print(df.shift(5))
print(df.shift(-5))

   Price
0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5   10.0
6   12.0
7    8.0
8   14.0
9   16.0
   Price
0   11.0
1    9.0
2   13.0
3   15.0
4   17.0
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN


In [24]:
import pandas as pd

df = pd.DataFrame({'Price': [10, 12, 8, 14, 16, 11, 9, 13, 15, 17]})

# print(df)
window_size=5
ma_minus = df['Price'].rolling(window=window_size).mean()
ma_plus = ma_minus.shift(-(window_size-1))

print(ma_plus[0]-ma_minus[4])


0.0
<class 'pandas.core.series.Series'>


In [4]:
#NOTE: , in the end convert to a function, with filepath and interval as parameters
#read value
df= pd.read_csv('./historical_data/Binance_ETHUSDT_d.csv',header=1) # second row is column names
interval = '1d'
print(df)
print(df.dtypes)




               Unix        Date   Symbol     Open     High      Low    Close  \
0     1680048000000  2023-03-29  ETHUSDT  1773.00  1827.99  1772.48  1793.07   
1     1679961600000  2023-03-28  ETHUSDT  1715.21  1795.73  1701.13  1772.99   
2     1679875200000  2023-03-27  ETHUSDT  1773.88  1780.48  1686.13  1715.22   
3     1679788800000  2023-03-26  ETHUSDT  1741.88  1801.00  1739.31  1773.88   
4     1679702400000  2023-03-25  ETHUSDT  1749.57  1763.23  1713.09  1741.88   
...             ...         ...      ...      ...      ...      ...      ...   
2046  1503273600000  2017-08-21  ETHUSDT   299.10   346.52   294.60   323.29   
2047  1503187200000  2017-08-20  ETHUSDT   289.41   300.53   282.85   299.10   
2048  1503100800000  2017-08-19  ETHUSDT   293.31   299.90   278.00   290.91   
2049  1503014400000  2017-08-18  ETHUSDT   302.00   311.79   283.94   293.96   
2050  1502928000000  2017-08-17  ETHUSDT   301.13   312.18   298.00   302.00   

        Volume ETH   Volume USDT  trade